In [250]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error as mse,r2_score as r2, mean_absolute_error as mae , make_scorer 
import sklearn.preprocessing as pre
from sklearn.model_selection import GridSearchCV, cross_val_score, TimeSeriesSplit
#Linear Models
import sklearn.linear_model as lm
#Trees - Non-Linear
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.ensemble import AdaBoostRegressor as ada

#NeuralNet - Piecewise Linear
from sklearn.neural_network import MLPRegressor as fnn

from main import RollingWindowSplit

In [251]:
surface = pd.read_csv('/home/student/surface/Realized Variance/DatasetSPX/Dataset30.csv')

In [252]:
surface

,index,date,-200.0,-195.0,-190.0,-185.0,-180.0,-175.0,-170.0,-165.0,...,vix80,vix,realVol30,realVol30_lagged,realVol30_baselag,var30,negVRP20,negVRP40,negVRP60,negVRP80
0,1996-01-04,19960104,0.136285,0.137428,0.142533,0.146466,0.149452,0.152037,0.153396,0.154342,...,0.001518,0.001561,0.000816,0.001088,0.001088,0.001696,-0.000215,-0.000327,-0.000387,-0.000430
1,1996-01-05,19960105,0.149338,0.150921,0.156802,0.161357,0.164838,0.167850,0.169486,0.170647,...,0.001415,0.001516,0.000799,0.000624,0.000624,0.001733,-0.000564,-0.000677,-0.000744,-0.000791
2,1996-01-08,19960108,0.126334,0.126139,0.129760,0.132430,0.134343,0.135980,0.136595,0.136910,...,0.001316,0.001413,0.000801,0.001346,0.000625,0.001899,0.000227,0.000120,0.000068,0.000030
3,1996-01-09,19960109,0.000000,0.119822,0.134540,0.147015,0.157548,0.166817,0.174007,0.180076,...,0.001969,0.001901,0.000984,0.001088,0.000628,0.001731,-0.000624,-0.000759,-0.000838,-0.000881
4,1996-01-10,19960110,0.172911,0.196304,0.222135,0.244068,0.262620,0.278939,0.291675,0.302442,...,0.002143,0.002211,0.001276,0.000624,0.000707,0.001490,-0.001104,-0.001309,-0.001443,-0.001519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,2017-11-09,20171109,0.202867,0.187733,0.195715,0.187120,0.196138,0.188586,0.195187,0.186414,...,0.000718,0.000906,0.000174,0.000380,0.000380,0.000507,-0.000043,-0.000194,-0.000280,-0.000338
5316,2017-11-10,20171110,0.221954,0.228170,0.215309,0.228243,0.214326,0.229455,0.212976,0.230103,...,0.000778,0.001048,0.000175,0.000347,0.000347,0.000508,-0.000103,-0.000269,-0.000366,-0.000431
5317,2017-11-13,20171113,0.246875,0.232928,0.238199,0.230959,0.240995,0.234005,0.241041,0.234609,...,0.000834,0.001087,0.000179,0.000363,0.000358,0.000508,-0.000112,-0.000294,-0.000401,-0.000471
5318,2017-11-14,20171114,0.245305,0.236642,0.240201,0.234513,0.239020,0.238946,0.242232,0.240686,...,0.000863,0.001104,0.000178,0.000379,0.000365,0.000494,-0.000116,-0.000302,-0.000412,-0.000483


In [253]:
surface.iloc[:,range(32,53,1)].head()

,-50.0,-45.0,-40.0,-35.0,-30.0,-25.0,-20.0,-15.0,-10.0,-5.0,...,5.0,10.0,15.0,20.0,25.0,30.0,35.0,40.0,45.0,50.0
0,0.356038,0.436592,0.501867,0.591655,0.655503,0.780144,0.868707,1.103014,1.404289,1.649329,...,1.214695,0.824629,0.574327,0.404571,0.232051,0.112947,0.000000,0.000000,0.000000,0.000000
1,0.314914,0.358148,0.425670,0.466838,0.524648,0.605602,0.720480,0.812035,1.026692,1.250084,...,1.483987,1.071744,0.744978,0.508356,0.346216,0.236822,0.153362,0.058284,0.000000,0.000000
2,0.295756,0.336799,0.365224,0.453709,0.526366,0.644405,0.716953,0.870466,1.077099,1.321225,...,1.208498,0.881416,0.580737,0.363094,0.231330,0.159816,0.077766,0.000000,0.000000,0.000000
3,0.431179,0.479081,0.576866,0.678638,0.720363,0.953343,1.066368,1.420370,1.464690,1.923692,...,1.678538,1.610239,1.087239,0.655229,0.670399,0.439168,0.255936,0.221609,0.144239,0.055413
4,0.562475,0.642685,0.731076,0.810379,0.948280,1.009774,1.177943,1.397184,1.636413,1.839707,...,1.553675,1.185673,0.923900,0.634686,0.432878,0.287902,0.177676,0.115572,0.103635,0.090031


### Estimate Model and Store Results

In [254]:
#Create an empty Dataframe to store Results
result1 = pd.DataFrame(columns=['VIX*2','Linear','Ridge','Lasso','Huber','TheilSen','RF','FNN'],index=[21,41,61,81])
result2 = pd.DataFrame(columns=['VIX*2','Linear','Ridge','Lasso','Huber','TheilSen','RF','FNN'],index=[21,41,61,81])
#Store results for our baseline model--> Use VIX*2 as a Proxy for Realized Variance
#result1.loc[81,'VIX*2'] = r2(surface.realVol30_lagged[1000:],surface.vix80[1000:])
#result1.loc[61,'VIX*2'] = r2(surface.realVol30_lagged[1000:],surface.vix60[1000:])
#result1.loc[41,'VIX*2'] = r2(surface.realVol30_lagged[1000:],surface.vix40[1000:])
#result1.loc[21,'VIX*2'] = r2(surface.realVol30_lagged[1000:],surface.vix20[1000:])
#result2.loc[81,'VIX*2'] = r2(surface.realVol30_lagged[1000:],surface.vix80[1000:])
#result2.loc[61,'VIX*2'] = r2(surface.realVol30_lagged[1000:],surface.vix60[1000:])
#result2.loc[41,'VIX*2'] = r2(surface.realVol30_lagged[1000:],surface.vix40[1000:])
#result2.loc[21,'VIX*2'] = r2(surface.realVol30_lagged[1000:],surface.vix20[1000:])


In [271]:
feature_set = [[2,83,'vix80',81],[12,73,'vix60',61],[22,63,'vix40',41],[32,53,'vix20',21]]
purge = 5
alpha1 = 100
alpha2 = 10000

In [265]:
'''LINEAR REGRESSION'''

Y1_true = []
Y1_hat  = []
Y2_true = []
Y2_hat  = []


for fset in feature_set:
    feature_start , feature_end , vix_code, result_index = fset
    
    
    
    for i,j in RollingWindowSplit().split_bounds(surface,1000,30, purge=purge):
        train_start, train_end = i[0], i[1]
        test_start, test_end = j[0], j[1]
        #Split Train Set
        X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
        
        Y1_train= surface.loc[train_start:train_end-1,'var30'].values   
        
        VIX_train = surface.loc[train_start:train_end-1,vix_code].values
        Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
        #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()


        #Split Test Set
        X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
        
        Y1_test= surface.loc[test_start:test_end-1,'var30'].values
        
        VIX_test = surface.loc[test_start:test_end-1,vix_code].values
        Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test



        '''REGRESSION 1'''
        #Intialize Linear Regressor
        regressor1 = lm.LinearRegression()

        #Fit the Linear Regressor
        regressor1.fit(X80_train,Y1_train)

        #Predict OOS
        Y1_true.extend(Y1_test)
        Y1_hat.extend(regressor1.predict(X80_test))


        '''REGRESSION 2'''
        #Intialize Linear Regressor
        regressor2 = lm.LinearRegression()

        #Fit the Linear Regressor
        regressor2.fit(X80_train,Y2_80_train)

        #Predict OOS
        Y2_true.extend(Y2_80_test  +  VIX_test)
        Y2_hat.extend(regressor2.predict(X80_test) + VIX_test)
       
    #Store results in the Results Dataframe
    result1.loc[result_index,'Linear'] = r2(Y1_true,Y1_hat)
    result2.loc[result_index,'Linear'] = r2(Y2_true,Y2_hat)
    

In [269]:
result2

,VIX*2,Linear,Ridge,Lasso,Huber,TheilSen,RF,FNN
21,0.28758,0.271559,0.357279,0.319619,NaN,NaN,NaN,NaN
41,0.334399,0.239854,0.380546,0.351854,NaN,NaN,NaN,NaN
61,0.358269,0.215102,0.390593,0.369452,NaN,NaN,NaN,NaN
81,0.372146,0.190108,0.394766,0.379924,NaN,NaN,NaN,NaN


In [270]:
result1

,VIX*2,Linear,Ridge,Lasso,Huber,TheilSen,RF,FNN
21,0.28758,0.271559,0.316926,-0.0128954,NaN,NaN,NaN,NaN
41,0.334399,0.239854,0.296759,-0.0128954,NaN,NaN,NaN,NaN
61,0.358269,0.215102,0.27614,-0.0128954,NaN,NaN,NaN,NaN
81,0.372146,0.190108,0.272177,-0.0128954,NaN,NaN,NaN,NaN


In [268]:
'''RIDGE REGRESSION'''

Y1_true = []
Y1_hat  = []
Y2_true = []
Y2_hat  = []

for fset in feature_set:
    feature_start , feature_end , vix_code, result_index = fset
    
    
    
    for i,j in RollingWindowSplit().split_bounds(surface,1000,30,purge=purge):
        train_start, train_end = i[0], i[1]
        test_start, test_end = j[0], j[1]
        #Split Train Set
        X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
        Y1_train= surface.loc[train_start:train_end-1,'var30'].values 
        VIX_train = surface.loc[train_start:train_end-1,vix_code].values
        Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
        #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()


        #Split Test Set
        X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
        Y1_test= surface.loc[test_start:test_end-1,'var30'].values
        VIX_test = surface.loc[test_start:test_end-1,vix_code].values
        Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test



        '''REGRESSION 1'''
        #Intialize Ridge Regressor
        regressor1 = lm.Ridge(alpha=alpha1)

        #Fit the Ridge Regressor
        regressor1.fit(X80_train,Y1_train)

        #Predict OOS
        Y1_true.extend(Y1_test)
        Y1_hat.extend(regressor1.predict(X80_test))


        '''REGRESSION 2'''
        #Intialize Ridge Regressor
        regressor2 = lm.Ridge(alpha=alpha2)

        #Fit the Ridge Regressor
        regressor2.fit(X80_train,Y2_80_train)

        #Predict OOS
        Y2_true.extend(Y2_80_test  +  VIX_test)
        Y2_hat.extend(regressor2.predict(X80_test) +  VIX_test)
   
    result1.loc[result_index,'Ridge'] = r2(Y1_true,Y1_hat)
    result2.loc[result_index,'Ridge'] = r2(Y2_true,Y2_hat)

In [260]:
'''VIX'''

Y1_true = []
Y1_hat  = []
Y2_true = []
Y2_hat  = []


for fset in feature_set:
    feature_start , feature_end , vix_code, result_index = fset
    
    
    
    for i,j in RollingWindowSplit().split_bounds(surface,1000,30, purge=purge):
        train_start, train_end = i[0], i[1]
        test_start, test_end = j[0], j[1]
        #Split Train Set
        X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
        Y1_train= surface.loc[train_start:train_end-1,'var30'].values   
        VIX_train = surface.loc[train_start:train_end-1,vix_code].values
        Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
        #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()


        #Split Test Set
        X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
        Y1_test= surface.loc[test_start:test_end-1,'var30'].values
        VIX_test = surface.loc[test_start:test_end-1,vix_code].values
        Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test



        '''REGRESSION 1'''
        #Intialize VIX Regressor
        #regressor1 = lm.LinearRegression()

        #Predict OOS
        Y1_true.extend(Y1_test)
        Y1_hat.extend(VIX_test)


        '''REGRESSION 2'''
        #Intialize VIX Regressor
        #regressor2 = lm.LinearRegression()

        #Predict OOS
        Y2_true.extend(Y2_80_test  +  VIX_test)
        Y2_hat.extend(VIX_test)
       
    #Store results in the Results Dataframe
    result1.loc[result_index,'VIX*2'] = r2(Y1_true,Y1_hat)
    result2.loc[result_index,'VIX*2'] = r2(Y2_true,Y2_hat)
    

In [261]:
'''LASSO REGRESSION'''

Y1_true = []
Y1_hat  = []
Y2_true = []
Y2_hat  = []


for fset in feature_set:
    feature_start , feature_end , vix_code, result_index = fset
    
    
    
    for i,j in RollingWindowSplit().split_bounds(surface,1000,30,purge=purge):
        train_start, train_end = i[0], i[1]
        test_start, test_end = j[0], j[1]
        #Split Train Set
        X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
        Y1_train= surface.loc[train_start:train_end-1,'var30'].values 
        VIX_train = surface.loc[train_start:train_end-1,vix_code].values
        Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
        #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()


        #Split Test Set
        X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
        Y1_test= surface.loc[test_start:test_end-1,'var30'].values
        VIX_test = surface.loc[test_start:test_end-1,vix_code].values
        Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test



        '''REGRESSION 1'''
        #Intialize Ridge Regressor
        regressor1 = lm.Lasso(alpha=alpha1)

        #Fit the Ridge Regressor
        regressor1.fit(X80_train,Y1_train)

        #Predict OOS
        Y1_true.extend(Y1_test)
        Y1_hat.extend(regressor1.predict(X80_test))


        '''REGRESSION 2'''
        #Intialize Ridge Regressor
        regressor2 = lm.Lasso(alpha=alpha2)

        #Fit the Ridge Regressor
        regressor2.fit(X80_train,Y2_80_train)

        #Predict OOS
        Y2_true.extend(Y2_80_test  +  VIX_test)
        Y2_hat.extend(regressor2.predict(X80_test) +  VIX_test)
   
    result1.loc[result_index,'Lasso'] = r2(Y1_true,Y1_hat)
    result2.loc[result_index,'Lasso'] = r2(Y2_true,Y2_hat)